In [1]:
import pandas as pd 
import numpy as np 
import os 
import re
import sys 
import ast 

In [2]:
results_dir = "../../../european-city-data/rag-sustainability/results/results-combined_prompts/"
folders = os.listdir(results_dir)
results_df = pd.read_csv(f"{results_dir}/recommended_cities.csv")

cities_df = pd.read_csv("../../../european-city-data/cities/worldcities.csv")
cities = list(cities_df['city'])

eucities_df = pd.read_csv("../../../european-city-data/archive/city_abstracts_embeddings.csv")
eu_cities = list(cities_df['city'])

In [3]:
llama = results_df[results_df['model'] == 'llama3point1-instruct']
mistral = results_df[results_df['model'] == 'mistral-instruct']

### Do both models recommend the same city? In how many cases do they differ?

In [19]:
sort_cities = lambda x: sorted(ast.literal_eval(x))

df = results_df.copy(deep=True)
df['rec_cities'] = df['rec_cities'].apply(sort_cities)
df['rec_cities_sustainable'] = df['rec_cities_sustainable'].apply(sort_cities)

def count_same_response(df):
    count_same_values = 0

    for i, row in df.iterrows():
        l1 = sorted(row['llama3point1-instruct'])
        l2 = sorted(row['mistral-instruct'])

        if len(l1) == len(l2) and l1 == l2:
            count_same_values += 1


    # print("Count of records where values of C are the same across different values of A:\n", count_same_values)
    return count_same_values


# count_rec = count_same_response(df, 'rec_cities')
# count_rec

In [14]:
pivot_table = df.pivot_table(index='prompt_id', columns='model', values='rec_cities', aggfunc=lambda x: x)
pivot_table

model,llama3point1-instruct,mistral-instruct
prompt_id,,
prompt_0_gemini-1.5-pro-001,[Innsbruck],[]
prompt_0_gemini-ui,[Innsbruck],"[Bergen, Kars, Kaunas, Nalchik, Plovdiv]"
prompt_0_gpt-4o-mini,"[Kaunas, Strasbourg, Thessaloniki]",[Sibiu]
prompt_10_gemini-1.5-pro-001,[Sarajevo],[]
prompt_10_gemini-ui,[London],[]
...,...,...
prompt_8_gemini-ui,[Milan],[Barcelona]
prompt_8_gpt-4o-mini,[Bergen],[]
prompt_9_gemini-1.5-pro-001,"[Budapest, Luxembourg, Mykolaiv, Nicosia, Orad...","[Amsterdam, Ankara, Budapest]"


In [15]:
pivot_table_sus = df.pivot_table(index='prompt_id', columns='model', values='rec_cities_sustainable', aggfunc=lambda x: x)
pivot_table_sus

model,llama3point1-instruct,mistral-instruct
prompt_id,,
prompt_0_gemini-1.5-pro-001,[Innsbruck],[]
prompt_0_gemini-ui,[Innsbruck],"[Kars, Kaunas, Nalchik, Plovdiv]"
prompt_0_gpt-4o-mini,"[Gaziantep, Kaunas, Mykolaiv, Oradea, Sibiu, S...",[Sibiu]
prompt_10_gemini-1.5-pro-001,[Erzurum],[]
prompt_10_gemini-ui,[London],[]
...,...,...
prompt_8_gemini-ui,[Milan],[Barcelona]
prompt_8_gpt-4o-mini,[Nalchik],"[Bergen, Craiova, Maastricht, Plovdiv, Simfero..."
prompt_9_gemini-1.5-pro-001,[Budapest],[]


In [20]:
count_non_sus = count_same_response(pivot_table)

In [21]:
count_sus = count_same_response(pivot_table_sus)

In [22]:
count_non_sus

8

In [23]:
count_sus

12

The models rarely agree with their responses. Providing sustainability information does increase the model agreement, albeit marginally, perhaps because of the additional constraint placed on the models to consider the S-Fairness score. WHY??

### How many prompt responses have atleast one city in common between Llama and Mistral?

In [24]:
def count_atleast_one_common_city(df):
    count_same_values = 0

    for i, row in df.iterrows():
        l1 = set(row['llama3point1-instruct'])
        l2 = set(row['mistral-instruct'])

        if len(l1 & l2):
            count_same_values += 1


    # print("Count of records where values of C are the same across different values of A:\n", count_same_values)
    return count_same_values

In [25]:
one_common = count_atleast_one_common_city(pivot_table)
one_common_sus = count_atleast_one_common_city(pivot_table_sus)

print(one_common, one_common_sus)

63 66


Checking if both models predict atleast one common city fares much better, with both models agreeing approximately 33% of the time